In [1]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip

--2021-07-04 22:32:59--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.67.16, 142.250.66.240, 172.217.167.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.67.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: 'horse-or-human.zip.1’

horse-or-human.zip. 100%[===================>] 142.65M  6.18MB/s    in 23s     

2021-07-04 22:33:22 (6.27 MB/s) - 'horse-or-human.zip.1’ saved [149574867/149574867]

--2021-07-04 22:33:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.67.16, 142.250.66.240, 172.217.167.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.67.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11

In [2]:
import os
import zipfile

local_train__zip = 'horse-or-human.zip'
local_validation__zip = 'validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_train__zip)
zip_ref.extractall('/tmp/horse-or-human')
zip_ref = zipfile.ZipFile(local_validation__zip)
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

2021-07-04 22:33:25.601454: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
model = keras.Sequential([
    keras.layers.Conv2D(16,(3,3), activation = tf.nn.relu, input_shape=(300,300,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(32,(3,3), activation = tf.nn.relu),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,(3,3), activation = tf.nn.relu),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128,(3,3), activation = tf.nn.relu),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,(3,3), activation = tf.nn.relu),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation = tf.nn.relu),
    keras.layers.Dense(units=1, activation = tf.nn.sigmoid)])

2021-07-04 22:33:26.345304: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-04 22:33:26.389351: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-04 22:33:26.389589: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 68 deviceMemorySize: 9.75GiB deviceMemoryBandwidth: 707.88GiB/s
2021-07-04 22:33:26.389604: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-04 22:33:26.391114: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-07-04 22:33:26.391139: I tensorflow/stream_executor/platform/def

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 128)       7

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

In [8]:
train_generator = train_datagen.flow_from_directory(
                    '/tmp/horse-or-human/',
                    target_size=(300,300),
                    batch_size=128,
                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
                    '/tmp/validation-horse-or-human/',
                    target_size=(300,300),
                    batch_size=128,
                    class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [9]:
history = model.fit_generator(train_generator,
         epochs=15,
         verbose=1,
         validation_data= validation_generator)

/home/arnab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-07-04 22:33:27.414445: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-04 22:33:27.431781: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3799900000 Hz


Epoch 1/15


2021-07-04 22:33:27.763922: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-07-04 22:33:28.133611: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2021-07-04 22:33:28.719117: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-07-04 22:33:29.056088: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-07-04 22:33:29.098556: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


9/9 [==============================] - 7s 607ms/step - loss: 0.7140 - acc: 0.5852 - val_loss: 0.6463 - val_acc: 0.5078
Epoch 2/15
9/9 [==============================] - 4s 495ms/step - loss: 0.5169 - acc: 0.7585 - val_loss: 0.6600 - val_acc: 0.8008
Epoch 3/15
9/9 [==============================] - 4s 493ms/step - loss: 0.2513 - acc: 0.9046 - val_loss: 0.8327 - val_acc: 0.8164
Epoch 4/15
9/9 [==============================] - 4s 495ms/step - loss: 0.1367 - acc: 0.9455 - val_loss: 2.0660 - val_acc: 0.7422
Epoch 5/15
9/9 [==============================] - 4s 495ms/step - loss: 0.1091 - acc: 0.9572 - val_loss: 1.9130 - val_acc: 0.8281
Epoch 6/15
9/9 [==============================] - 4s 495ms/step - loss: 0.0535 - acc: 0.9844 - val_loss: 1.2445 - val_acc: 0.8359
Epoch 7/15
9/9 [==============================] - 4s 494ms/step - loss: 0.0219 - acc: 0.9893 - val_loss: 2.1552 - val_acc: 0.8164
Epoch 8/15
9/9 [==============================] - 4s 488ms/step - loss: 0.0138 - acc: 0.9932 - val_lo